# Data manipulation with dplyr

In [ ]:
# Some initial setup
options(digits = 3, repr.matrix.max.rows = 6)

# load tidyverse, which includes dplyr and ggplot
library("tidyverse")

The [tidyverse](https://www.tidyverse.org) is a collection of actively-developed `R` packages that follow a certain principle of writing code.
Here, we will primarily focus on `dplyr`, which is the part of `tidyverse` which deals with data manipulation.

First, we will start by reading some data. 
There are many functions in `tidyverse` (and `R` in general) for reading various data formats into the `R` environment as a table of data (or, `data.table`).
For example, either `read_csv` (from `tidyverse`) or `read.csv` (base `R`) can be used to read a `csv` file (e.g., a file where each line represents a row in the table, and the columns are separated by a `,`).

For this tutorial, we will be using the [titanic](https://www.kaggle.com/c/titanic/data) passenger data from Kaggle.
The schema of the data are being described in the following table.

**Variable**|**Definition**|**Key**
:-----:|:-----:|:-----:
survival|Survival|0 = No, 1 = Yes
pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd
sex|Sex| 
Age|Age in years| 
sibsp|# of siblings / spouses aboard the Titanic| 
parch|# of parents / children aboard the Titanic| 
ticket|Ticket number| 
fare|Passenger fare| 
cabin|Cabin number| 
embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampton

To get started, let's first load and take a look at the data:

In [ ]:
# load the csv data
# column types can be inferred automatically, but can also be specified with col_types argument
passengers <- read_csv(
    "titanic.csv",
    col_types = cols(.default = "d", Name='c', Sex='f', Ticket='c', Embarked='c', Cabin='f'))
# cleaning some data
passengers <- passengers %>% drop_na(Embarked, Age)


# Summary of each column
summary(passengers)

## Introduction to `dplyr` verbs

In the world of `dplyr`, a _verb_ is a function that

* takes a data frame as its first argument, and
* returns another data frame as a result

Any function that meets this criteria, even if it's not necessarily a function in the `dplyr` package, can be considered a _verb_.

For example, the `head()` function in `R`, if applied to a data frame, will return a first `n` rows of a data frame.

In [ ]:
head(passengers, n = 3)  # Return a data frame that consists of the first three rows of titanic

In this sense, the base `R` function `head()` is a verb.

The **core idea** of `dplyr` is that a vast majority of data manipulation needs can be satisfied through a combination of five verbs.

verb                 | action
-------------------- | ---------
`filter(df, ...)`    | select a subset of _rows_ by some specified condition
`select(df, ...)`    | select a subset of _columns_
`mutate(df, ...)`    | create a _new column_ (often as a function of existing columns)
`arrange(df, ...)`   | reorder (sort) _rows_ according to values of specific _columns_
`summarize(df, ...)` | aggregate and reduce a vector (column) to a single value

We will explore each of these verbs (and some additional variations within each category) below.

### Selecting rows (1/5)

The `filter(df, ...)` verb is used to select a subset of _rows_ that satisfy the conditions specified in `...`. 
The conditions must be written in a form that would evaluate to either `TRUE` or `FALSE`.

For example, if we want a data frame of passengers in the 3rd clas:

In [ ]:
filter(passengers, Pclass == 3)

Multiple conditions can also be specified.

For example, if we want a data frame of all passengers in the 3rd class that are older than 50 years old:

In [ ]:
filter(passengers, Pclass == 3, Age > 50)

By default, `filter()` will combine multiple conditions as `AND` operations.
In the example above, we are returned passengers where `Pclass == 3` _**AND**_ `Age < 18`.

We can specify an OR condition by using the `|` operator.

For example, if we want passengers in the 3rd class who are either younger than 18 years old or older than 65 years old:

In [ ]:
filter(passengers, Pclass == 3, Age > 50 | Age < 18)

Use the `%in%` operator to filter to values that match a collection of values. 

For example, suppose we want to look at passengers embarked on Queenstown (`Q`) and Southampton (`S`)

In [ ]:
filter(passengers, 
       Embarked %in% c("Q", "S"))

Finally, use `!` to negate any condition. 

For example, if we wanted to find all passengers who did NOT embark on Queenstown (`Q`) and Southampton (`S`)

In [ ]:
filter(passengers, 
       !(Embarked %in% c("Q", "S")))

### Exercise: `filter()`
Find all pasengers with 2 siblings and 2 parents aboard.

In [ ]:
# YOUR CODE HERE
filter(passengers, SibSp == 2, Parch == 2)

### Selecting columns (2/5)

Use `select(df, ...)` to either specify which columns to select,

In [ ]:
select(passengers, Survived, Pclass, Sex, Age)

or to specify which columns to exclude, using `-`.

In [ ]:
select(passengers, -Name, -SibSp, -Parch)

`dplyr` also provides some useful helper functions to `select()` columns that match specific criteria.

* `starts_with(x)`: match column names that start with `x`
* `ends_with(x)`: match column names that end with `x`
* `contains(x)`: match column names that contain `x`
* `matches(x)`: match column names that match (the regular expression) `x`

where `x` is a string (in either single- or double-quotes).

For example, if we want all the columns that start with letter `s`:

In [ ]:
select(passengers, starts_with('S'))

You can see the documentation for `select()` for details.
In general, for any `R` function, you can pull-up the documentation (if one exists) by running `?` followed by the function name.
For example, to see the documentation for `select()` as provided in the `dplyr` package, run:

In [ ]:
?dplyr::select

### Create new columns (3/5)

Use `mutate(df, ...)` to create new columns, usually as a function of existing columns.

Suppose we wish to create a column called `no_sib`, indicating whether the passenger has no sibling aboard.
We would write,

In [ ]:
mutate(passengers, no_sib = (SibSp == 0))

We note that within a single `mutate()` function, you can refer to the new columns you've created in the previous argument.


### Exercise: `mutate()`
Create a column `num_family` representing the number of family members on board for each passenger (the sum of sibling number and parent number)

In [ ]:
# YOUR CODE HERE
mutate(passengers, num_family = SibSp + Parch)

### Sorting (4/5)

Use `arrange(df, ...)` to reorder the rows of a data frame by the value of specified columns.
Multiple conditions are arranged from left to right.

In [ ]:
arrange(passengers, Pclass, Name)

Use `desc()` around columns that you want to sort in `desc`ending order.

In [ ]:
arrange(passengers, Pclass, desc(Name))

### Aggregating (5/5)

Use `summarize(df, ...)` to aggregate multiple rows into a single row. Unlike `mutate()`, function that are used in `summarize()` must return a single value (i.e., "aggregate" the provided vector)

For example, to find the min, mean, and max age of all passengers,

In [ ]:
summarize(passengers,
          min_height = min(Age),
          avg_height = mean(Age),
          max_height = max(Age))

`dplyr` also provides a special function `n()` which will evaluate to the number of rows within a `dplyr` verb.

For example, to count how many passengers (rows) there are in our dataset in total, 

In [ ]:
summarize(passengers, N = n())

### Exercise: `summarize()`
Calculate the mean, median and standard deviation of `Fare`.
(hint: using `median` and `sd` function for median and standard deviation respectively) 

In [ ]:
# YOUR CODE HERE
summarize(passengers,
          mean = mean(Fare),
          median = median(Fare),
          sd = sd(Fare))

### Grouping (Split-apply-combine)

Now that we've covered the five core verbs, we should be able to manipulate data to our heart's desires, yes?

Then, how about:

* The number of passengers for each `Sex`?
* Survival rates in each `Pclass`?
* Number of each `Pclass` for passengers embarked on each port?

As an example, let's just consider the number of passengers for each `Sex`

A natural, but _**tedious**_ way to compute this would look something like this:

In [ ]:
# NOTE: Code in this cell is intended to be an example of a BAD way to compute this.
#       This is purely for illustrative purposes, and should NEVER EVER be re-used, in any context.
male_passengers <- filter(passengers, Sex == "male")
female_passengers <- filter(passengers, Sex == "female")

N_males <- summarize(male_passengers, N = n())
N_females <- summarize(female_passengers, N = n())

c(males = N_males$N, females = N_females$N)

This style of code can easily get out of hand, and would be a nightmare to maintain! 
(e.g., what happens if we get a new dataset where we want to count the number of rows for 8 different categories rather than just 2 categories like `male` and `female`?)

As horrible as the above code is, it is useful in highlighting a common pattern that emerges when manipulating data:

1. **Split**: The data are split into smaller pieces of data, according to one (or more) column. 
   In this case, we've split the data by the `Sex` column.
1. **Apply**: Some operation is applied to each of the smaller pieces.
   In this case, we've simply counted the number of rows of each piece using `summarize()` and `n()`.
1. **Combine**: The results of the previous **apply** are combined to some final data structure.
   In the above case, for simplicity, we've combinded the result as a vector; but in practice we usually want to keep everything in the form of a data frame.

This pattern in data manipulation is so common, that there is a `dplyr` verb for it. 
This is the `group_by` verb.

On it's own, `group_by` makes no visible changes to a data frame, other than marking the data frame as being "grouped".
The difference is only made apparent when we apply some other verb to a grouped data frame.

Note that none of the `dplyr` verbs make any changes to the original data frame! This is very intentional. 
So, for now, we need to save the "grouped" data as a new variable for our changes to have effect (but we'll see a more convenient approach to this later).

In [ ]:
passengers_by_gender <- group_by(passengers, Sex)

# Note that the two data frames, on the surface, seem identical.
passengers
passengers_by_gender

In [ ]:
# But we can see a difference when applying, for example, a summarize
summarize(passengers, N = n())

In [ ]:
summarize(passengers_by_gender, N = n())

As shown in the simple example above, when a `dplyr` verb is applied to a "grouped" data frame,
`dplyr` internally **splits**-**applies**-and **combines** the data, finally returning results for
_each of the unique values that are found in the columns by which the data frame is grouped_.

This can be a lot to process, if it's the first time you've seen this. 
But once you get used to it (via trying a bunch of manipulation tasks and seeing some more examples), you'll find it extremely convenient and powerful.

Let's try answering the other questions we started this section with.

* Survival rates in each `Pclass`?

In [ ]:
passengers_by_class <- group_by(passengers, Pclass)

summarize(passengers_by_class, 
          survival_rate = mean(Survived))

* Number of each `Pclass` for passengers embarked on each port?

In [ ]:
passengers_by_port <- group_by(passengers, Embarked, Pclass)

summarize(passengers_by_port, 
          N = n())

### Exercise: `group_by()`

Calculate the mean fare and survival rate for male and female passengers respectively

In [ ]:
# YOUR CODE HERE
gender_group <- group_by(passengers, Sex)
summarize(gender_group, 
          mean_fare = mean(Fare),
          survival_rate = mean(Survived)
         )

### Multiple (chained) operations

As we've breifly seen above, we would often like to apply multiple operations (verbs) to a data frame.
However, by design, verbs do not save intermediate changes to the original data frame, so for each operation we would have to assign the result to a new data frame.

Even for a reasonable number of operations, this can get quite messy (i.e., we'd end up with so many names and data frames that we only use as intermediate steps).

Consider the following query:

* For each port, what is the proportion of each class?

We can think of finding the answer in multiple steps:

1. group by `Embarked` and `Pclass`
1. find the number of passengers for each of the groups in the previously grouped data frame
1. with the computed number of passengers for each class-port pair, re-group by only `Embarked`
1. create a new column which computes the proportion of counts for each row, over the sum of rows

Using the current method of saving all intermediate results, the implementation would look something like this:

In [ ]:
# NOTE: Code in this cell is intended to be an example of a BAD implementation.
#       While this implementation is acceptable, it is intended for illustrative purposes, 
#       and is best avoided.  
passengers_by_port_and_class <- group_by(passengers, Embarked, Pclass)
counts_by_port_and_class <- summarize(passengers_by_port_and_class, N = n())
regroup_by_port <- group_by(counts_by_port_and_class, Embarked)
mutate(regroup_by_port, prop = N / sum(N))

The above code is bad for a multiple reasons. Among others, it's

* creating a lot of unnecessary intermediate results that will not be used again
* difficult to read, if you don't already know what the end goal is (e.g., you eyes have to wander left-to-right-to-left a few times to see what's going on)

A sophisticated, yet quite simple, solution to this problem is the introduction of `%>%`, also called the "pipe operator".

`%>%` is a _binary operator_ (much like `+` or `-`) which, in words, takes the result of the left-hand side, and uses it as the first argument on the right hand side. 
This may be confusing at first, but might make more sense in the context of `dplyr` _verbs_. Recall, a _verb_ in `dplyr` is any function that _returns a data frame_ (LHS) and _takes a data frame as its first argument_ (RHS).

If further notation is helpful, one could also write the `%>%` as
```
f(x) %>% g(y) = g(f(x), y)
```
which, depending on your background, could be clarifying or more confusing.

What this means from a practical standpoint, however, is that we no longer need to _save_ intermediate results just to use them in the next verb. 
Instead, we can use `%>%` to send results from a verb down a "pipe" to the next verb.
Consider our previous example, which involved four verbs, with three intermediate steps.
Using `%>%`, the same result can now be achieved in a (conceptually) single line:

In [35]:
passengers %>% 
    group_by(Embarked, Pclass) %>%
    summarize(N = n()) %>%
    group_by(Embarked) %>%
    mutate(prop = N / sum(N))

Embarked,Pclass,N,prop
<chr>,<dbl>,<int>,<dbl>
C,1,74,0.569
C,2,15,0.115
C,3,41,0.315
⋮,⋮,⋮,⋮
S,1,108,0.195
S,2,156,0.282
S,3,290,0.523


Note the intentional style of (1) starting from the data frame (instead of a verb that explicitly includes the data frame) and (2) keeping each verb on its own line. 
This not only makes it easier to read, but also easier to maintain and modify.

In [36]:
head(passengers, 1)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S


In [59]:
passengers %>%
  mutate(age_group = 10 * floor(Age / 10)) %>%
  group_by(Sex, age_group) %>%
  summarize(num_pass = count(), survival_rate = mean(Survived))
# summarize(, min_age_brack = min(age_bracket))

ERROR: Error in summarise_impl(.data, dots): Evaluation error: argument "x" is missing, with no default.


### Exercise: Introduction to `dplyr`
1. Create a column `age_group` that groups each passenger's age down to the 10s.
1. Calculate the number of passengers and survival rate for each (`Sex`, `age_group`) pair

What patterns do you observe?

(hint1: function `floor` takes in x and returns the largest integers not greater than the corresponding elements of x, e.g., `floor(1.9)` will return `1`)

(hint2: you can run `options(repr.matrix.max.rows = 20)` to display more rows at a time)


In [ ]:
# YOUR CODE HERE
options(repr.matrix.max.rows = 20)

passengers  %>% 
    mutate(age_group = floor(Age / 10) * 10) %>%
    group_by(Sex, age_group) %>%
    summarize(
        N = n(),
        survival_rate = mean(Survived)
    )

## Ending notes

There are many, MANY more verbs that we simply did not have the time to cover here, but are immensely useful. 
Some examples are:

* `rename(df, ...)`: rename columns
* `slice(df, ...)`: select rows of a data frame by index, instead of some condition
* `top_n(df, N, col)`: retrieve the top N rows for values in some specified column

You are highly recommended to explore more. One great resource for learning about `tidyverse` and using it to work with data is Hadley Wickham's online book: [R for Data Science](https://r4ds.had.co.nz/). 

Hadley Wickham is also the original author for many of the packages in `tidyverse`. In fact, in the "early days" (circa 2016), before the word `tidyverse` was created, the collection of Hadley Wickham's `R` packages were unofficially referred to as the `hadleyverse`, until [Hadley announced tidyverse and explicitly asked people to stop calling it the hadleyverse](https://twitter.com/hadleywickham/status/774008060549312512?lang=en).